<a href="https://colab.research.google.com/github/sharwanbagaria03/Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("data.csv")
print(data.head())

   Unique_ID Category                           Review_Header  \
0     136040  smartTv                                Nice one   
1     134236   mobile  Huge battery life with amazing display   
2     113945    books                              Four Stars   
3     168076  smartTv                            Nice quality   
4     157302    books                               Nice book   

                                         Review_text  Rating Own_Rating  
0                                         I liked it       5   Positive  
1  I bought the phone on Amazon and been using my...       5   Positive  
2  Awesome book at reasonable price, must buy ......       4   Positive  
3                                               good       5   Positive  
4  The book is fine,not bad,contains nice concept...       3    Neutral  


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60889 entries, 0 to 60888
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unique_ID      60889 non-null  int64 
 1   Category       60889 non-null  object
 2   Review_Header  60884 non-null  object
 3   Review_text    60857 non-null  object
 4   Rating         60889 non-null  int64 
 5   Own_Rating     60889 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.8+ MB


In [ ]:
data['review'] = data['Review_Header'].astype(str) + ': ' + data['Review_text'].astype(str)

In [ ]:
data = data.drop(columns=['Unique_ID', 'Category', 'Rating', 'Review_Header', 'Review_text'])

In [ ]:
positive_data = data[data['Own_Rating'] == 'Positive']
negative_neutral_data = data[data['Own_Rating'].isin(['Negative', 'Neutral'])]
positive_sampled_data = positive_data.sample(n=10000, random_state=42)
data = pd.concat([negative_neutral_data, positive_sampled_data])
data = data.reset_index(drop=True)

In [ ]:
data.shape

(23453, 2)

In [ ]:
data.head()

,Own_Rating,review
0,Neutral,"Nice book: The book is fine,not bad,contains n..."
1,Negative,Need more good quality: The company should giv...
2,Negative,"Cool product, bad service: Product is good as ..."
3,Negative,Product not working: Product not working. Carr...
4,Negative,Not purchase: Not purchase if you are miui use...


In [ ]:
reviews = data['review']
sentiments = list(data['Own_Rating'])

### Clean Text data

In [ ]:
import nltk
nltk.download('wordnet')
import re
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
CLEANR = re.compile('<.*?>')

def clean(review):
    review = re.sub(CLEANR, '', review) # remove html tags
    review = re.sub('[^a-zA-Z ]', '', review)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(i) for i in review]
    return ' '.join(review)

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
reviews = reviews.apply(clean)
reviews[:10]

,review
0,nice book the book is finenot badcontains nice...
1,need more good quality the company should give...
2,cool product bad service product is good a exp...
3,product not working product not working carrie...
4,not purchase not purchase if you are miui user...
5,worst product worst product just becz of rain ...
6,thing considered before buyingthese problem wi...
7,quality of page not good got pirated copy a th...
8,nice performance nice performance but battery ...
9,good good


In [ ]:
y = pd.get_dummies(sentiments)['Positive']

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(list(reviews), y, test_size=0.2, random_state=0)

### Setup Transformers

In [ ]:
import transformers
import tensorflow as tf

In [ ]:
# Tokenization
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
train_encodings = tokenizer(x_train,
                            truncation=True,
                            padding=True)

val_encodings = tokenizer(x_test,
                            truncation=True,
                            padding=True)

In [ ]:
# convert to tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_test
))

### Load pre-trained model

In [ ]:
# Load Model
from transformers import TFDistilBertForSequenceClassification, AutoTokenizer
import tensorflow as tf

# Load pre-trained model and tokenizer, change num_labels to 1
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=1)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

### Train model

In [ ]:
# Compile the model with BinaryCrossentropy loss and from_logits=True
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
# For num_labels=1, you should use sigmoid activation and binary cross-entropy loss
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Train the model
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=2,
          validation_data=val_dataset.shuffle(100).batch(16))


Epoch 1/2
1173/1173 [==============================] - 1118s 933ms/step - loss: 0.2983 - accuracy: 0.8779 - val_loss: 0.2871 - val_accuracy: 0.8919
Epoch 2/2
1173/1173 [==============================] - 1092s 931ms/step - loss: 0.2285 - accuracy: 0.9096 - val_loss: 0.2990 - val_accuracy: 0.8915


In [ ]:
# train for more 2 epochs
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=2,
          validation_data=val_dataset.shuffle(100).batch(16))

In [ ]:
model.save_pretrained("/content/output")

In [ ]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("/content/output")

Some layers from the model checkpoint at /content/output were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/output and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
predictions = model.predict(val_dataset.batch(16)).logits
predicted_probs = tf.sigmoid(predictions)

294/294 [==============================] - 89s 303ms/step


In [ ]:
# Define the classification function
def classify_predictions(predictions, threshold_positive=0.7, threshold_neutral=0.3):
    predicted_probs = tf.sigmoid(predictions)

    predictions_class = []
    for prob in predicted_probs:
        if prob >= threshold_positive:
            predictions_class.append('Positive')
        elif prob <= threshold_neutral:
            predictions_class.append('Negative')
        else:
            predictions_class.append('Neutral')

    return predictions_class

In [ ]:
classified_predictions = classify_predictions(predictions)
results_df = pd.DataFrame({'Review': x_test, 'True Sentiment': y_test, 'Predicted Sentiment': classified_predictions})

In [ ]:
# Save the results to a CSV file
results_df.to_csv('/content/predictions.csv', index=False)